In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [4]:
# 엑셀 파일 경로
file_path1 = r'C:\Users\82108\OneDrive\바탕 화면\investment\investment_strategy\DATA\toptier_company_HS_Code_2022_VER1.xlsx'
file_path2 = r"C:\Users\82108\OneDrive\바탕 화면\investment\investment_strategy\DATA\korea_hscode_list.csv"

# 시트 이름이 있다면 명시, 없으면 첫 시트 사용
df = pd.read_excel(file_path1)

# 전처리 예시 (필요 시 수정하세요)
df.columns = df.columns.str.strip()              # 컬럼 이름 공백 제거
df = df.dropna(how='all')                        # 모든 값이 NaN인 행 제거
df = df.drop_duplicates() 

# 1. hs_code가 문자열로 변환된 후, 길이가 6 이상인 행만 추출
df['hs_code'] = df['hs_code'].astype(str)
df_filtered = df[df['hs_code'].str.len() >= 6].copy()

# 2. 앞 6자리만 추출하여 새 컬럼 'hs_code_6d' 생성
df_filtered['hs_code_6d'] = df_filtered['hs_code'].str[:6]

In [5]:
# 시트 이름이 있다면 명시, 없으면 첫 시트 사용
hs_code_500 = pd.read_csv(file_path2)

# 두 시리즈를 모두 문자열로 변환
hs_code_from_csv = hs_code_500['HS_Code'].astype(str)
hs_code_from_excel = df_filtered['hs_code_6d'].astype(str)

# 두 시리즈를 합치고 중복 제거
combined_hs_codes = pd.concat([hs_code_from_csv, hs_code_from_excel]).drop_duplicates().reset_index(drop=True)

# 새로운 DataFrame 생성
hs_code_df = pd.DataFrame({'hs_code': combined_hs_codes})

# 결과 확인
print(hs_code_df.head())
print(f"총 {len(hs_code_df)}개의 고유 hs_code가 있습니다.")

  hs_code
0  121120
1  121221
2  170199
3  190230
4  190590
총 567개의 고유 hs_code가 있습니다.


In [6]:
# DB 접속 정보 설정
db_info = {
    'user': 'stox7412',           # 예: 'root'
    'password': 'Apt106503!~',   # 예: '1234'
    'host': '192.168.0.230',           # 예: 'localhost'
    'port': '3307',                # 예: '3306'
    'database': 'investar'    # 예: 'trade_data'
    
}

# SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
)

# 테이블 이름 지정
table_name = 'target_hs_code'

# DB에 업로드 (기존 테이블 덮어씀 → 'replace', 추가는 'append')
hs_code_df.to_sql(name=table_name, con=engine, index=False, if_exists='replace')

print(f"✅ {table_name} 테이블에 {len(hs_code_df)}개의 행이 업로드되었습니다.")

✅ target_hs_code 테이블에 567개의 행이 업로드되었습니다.


In [9]:
### 주요 수출기업의 주력상품 HS 코드 정보를 입력하고 출력하는 파이썬 코드를 만들다 

# SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
)

# 테이블 이름 지정
table_name = 'hs_code_by_kr_monster_company'

# DB에 업로드 (기존 테이블 덮어씀 → 'replace', 추가는 'append')
df_filtered.to_sql(name=table_name, con=engine, index=False, if_exists='replace')

print(f"✅ {table_name} 테이블에 {len(hs_code_df)}개의 행이 업로드되었습니다.")

✅ hs_code_by_kr_monster_company 테이블에 567개의 행이 업로드되었습니다.


In [8]:
df_filtered 

,hs_code,품목명,Code,Name,hs_code_6d
15,392321,엘씨디카세트,A089980,상아프론테크,392321
16,470710,폐골판지,A016590,신대양제지,470710
17,480511,골심지,A016590,신대양제지,480511
18,560394,폴리우레탄합성피혁,A035150,백산,560394
19,630293,극세사클리너,A065950,웰크론,630293
...,...,...,...,...,...
195,9031809091,반도체오버레이계측기,A322310,오로스테크놀로지,903180
196,9401309000,디지털구강내엑스선영상획득장치,A228850,레이언스,940130
197,9403309000,사무용의자,A016800,퍼시스,940330
198,9405409000,선박용형광등기구,A108380,대양전기공업,940540
